In [2]:
import sqlite3
import streamlit as st


In [3]:
# Inicializar la base de datos
def init_db():
    conn = sqlite3.connect("biblioteca.db")
    c = conn.cursor()

    c.execute("""
    CREATE TABLE IF NOT EXISTS libros (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        titulo TEXT,
        autor TEXT,
        tema TEXT,
        anio INTEGER,
        solucionario BOOLEAN,
        ruta TEXT,
        notas TEXT
    )
    """)

    c.execute("""
    CREATE TABLE IF NOT EXISTS tags (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT UNIQUE
    )
    """)

    c.execute("""
    CREATE TABLE IF NOT EXISTS libro_tags (
        libro_id INTEGER,
        tag_id INTEGER,
        FOREIGN KEY(libro_id) REFERENCES libros(id),
        FOREIGN KEY(tag_id) REFERENCES tags(id),
        UNIQUE(libro_id, tag_id)
    )
    """)

    conn.commit()
    conn.close()

In [4]:
# Agregar libro
def agregar_libro(datos, tags):
    conn = sqlite3.connect("biblioteca.db")
    c = conn.cursor()

    c.execute("""
        INSERT INTO libros (titulo, autor, tema, anio, solucionario, ruta, notas)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, datos)
    libro_id = c.lastrowid

    for tag in tags:
        c.execute("INSERT OR IGNORE INTO tags (nombre) VALUES (?)", (tag,))
        c.execute("SELECT id FROM tags WHERE nombre = ?", (tag,))
        tag_id = c.fetchone()[0]
        c.execute("INSERT OR IGNORE INTO libro_tags (libro_id, tag_id) VALUES (?, ?)", (libro_id, tag_id))

    conn.commit()
    conn.close()

In [5]:
# Buscar libros por tag
def buscar_libros_por_tag(tag):
    conn = sqlite3.connect("biblioteca.db")
    c = conn.cursor()

    c.execute("""
        SELECT libros.titulo, libros.autor, libros.tema, libros.anio, libros.ruta, libros.solucionario
        FROM libros
        JOIN libro_tags ON libros.id = libro_tags.libro_id
        JOIN tags ON tags.id = libro_tags.tag_id
        WHERE tags.nombre = ?
    """, (tag,))

    resultados = c.fetchall()
    conn.close()
    return resultados

In [7]:
st.title("Biblioteca de Libros de Física y Astrofísica")

2025-07-12 00:36:37.334 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 00:36:37.337 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator(_form_data=FormData(form_id='agregar_libro'))

In [1]:
# Requiere: streamlit, sqlite3
# Ejecuta con: streamlit run nombre_del_archivo.py

import sqlite3
import streamlit as st

# Inicializar la base de datos
def init_db():
    conn = sqlite3.connect("biblioteca.db")
    c = conn.cursor()

    c.execute("""
    CREATE TABLE IF NOT EXISTS libros (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        titulo TEXT,
        autor TEXT,
        tema TEXT,
        anio INTEGER,
        solucionario BOOLEAN,
        ruta TEXT,
        notas TEXT
    )
    """)

    c.execute("""
    CREATE TABLE IF NOT EXISTS tags (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT UNIQUE
    )
    """)

    c.execute("""
    CREATE TABLE IF NOT EXISTS libro_tags (
        libro_id INTEGER,
        tag_id INTEGER,
        FOREIGN KEY(libro_id) REFERENCES libros(id),
        FOREIGN KEY(tag_id) REFERENCES tags(id),
        UNIQUE(libro_id, tag_id)
    )
    """)

    conn.commit()
    conn.close()

# Agregar libro
def agregar_libro(datos, tags):
    conn = sqlite3.connect("biblioteca.db")
    c = conn.cursor()

    c.execute("""
        INSERT INTO libros (titulo, autor, tema, anio, solucionario, ruta, notas)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, datos)
    libro_id = c.lastrowid

    for tag in tags:
        c.execute("INSERT OR IGNORE INTO tags (nombre) VALUES (?)", (tag,))
        c.execute("SELECT id FROM tags WHERE nombre = ?", (tag,))
        tag_id = c.fetchone()[0]
        c.execute("INSERT OR IGNORE INTO libro_tags (libro_id, tag_id) VALUES (?, ?)", (libro_id, tag_id))

    conn.commit()
    conn.close()

# Buscar libros por tag
def buscar_libros_por_tag(tag):
    conn = sqlite3.connect("biblioteca.db")
    c = conn.cursor()

    c.execute("""
        SELECT libros.titulo, libros.autor, libros.tema, libros.anio, libros.ruta, libros.solucionario
        FROM libros
        JOIN libro_tags ON libros.id = libro_tags.libro_id
        JOIN tags ON tags.id = libro_tags.tag_id
        WHERE tags.nombre = ?
    """, (tag,))

    resultados = c.fetchall()
    conn.close()
    return resultados

# Streamlit UI
st.title("📚 Biblioteca de Libros de Física y Astrofísica")

init_db()

menu = st.sidebar.selectbox("Menú", ["Agregar libro", "Buscar por tag"])

if menu == "Agregar libro":
    with st.form("agregar_libro"):
        titulo = st.text_input("Título")
        autor = st.text_input("Autor")
        tema = st.text_input("Tema principal")
        anio = st.number_input("Año", min_value=1800, max_value=2100, step=1)
        solucionario = st.checkbox("¿Tiene solucionario?")
        ruta = st.text_input("Ruta del archivo")
        notas = st.text_area("Notas adicionales")
        tags_input = st.text_input("Tags (separados por comas)")
        enviar = st.form_submit_button("Guardar")

        if enviar:
            tags = [t.strip().lower() for t in tags_input.split(",") if t.strip()]
            datos = (titulo, autor, tema, anio, solucionario, ruta, notas)
            agregar_libro(datos, tags)
            st.success("✅ Libro agregado correctamente")

elif menu == "Buscar por tag":
    tag = st.text_input("Introduce un tag (ej. relatividad)")
    if tag:
        resultados = buscar_libros_por_tag(tag.lower())
        if resultados:
            for r in resultados:
                st.write(f"**Título:** {r[0]}\n\n**Autor:** {r[1]}\n\n**Tema:** {r[2]}\n\n**Año:** {r[3]}\n\n**Ruta:** {r[4]}\n\n**Solucionario:** {'Sí' if r[5] else 'No'}")
                st.markdown("---")
        else:
            st.warning("No se encontraron libros con ese tag.")


2025-07-12 00:33:50.827 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 00:33:50.989 
  command:

    streamlit run /home/holman/miniconda3/envs/cosmos/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-12 00:33:50.990 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 00:33:51.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 00:33:51.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 00:33:51.001 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 00:33:51.002 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 00:33:51.002 S